In [1]:
library(nichenetr)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
## We load the networks generated in the ParameterOptimization script
lr_network_Omnipath = readRDS("NNET_Custom/lig_rec_sources.rds")
sig_network_Omnipath = readRDS("NNET_Custom/sig_Network.rds")
gr_network_Omnipath = readRDS("NNET_Custom/gr_Network.rds")

# Construct NicheNet’s ligand-target model from unoptimized data source weights


In [3]:
## The interactions are weighted only based in the number of data sources that
## report them
All_sources <- unique(c(lr_network_Omnipath$source,
    sig_network_Omnipath$source, gr_network_Omnipath$source))

In [4]:
All_sources

[1] "CellTalkDB"                  "HPRD"                       
  [3] "TRIP"                        "Baccin2019"                 
  [5] "CellCall"                    "CellChatDB"                 
  [7] "CellPhoneDB"                 "CellPhoneDB_Cellinker"      
  [9] "Cellinker"                   "DLRP_Cellinker"             
 [11] "DLRP_talklr"                 "EMBRACE"                    
 [13] "Fantom5_LRdb"                "HPMR_Cellinker"             
 [15] "HPMR_LRdb"                   "HPMR_talklr"                
 [17] "HPRD_LRdb"                   "HPRD_talklr"                
 [19] "ICELLNET"                    "KEGG-MEDICUS"               
 [21] "Kirouac2010"                 "LRdb"                       
 [23] "NetPath"                     "Ramilowski2015"             
 [25] "Ramilowski2015_Baccin2019"   "SIGNOR"                     
 [27] "STRING_talklr"               "SignaLink3"                 
 [29] "UniProt_LRdb"                "Wang"                       
 [31] "connectomeDB2020"            "iTALK"                      
 [33] "talklr"                      "SPIKE"                      
 [35] "CA1"                         "DIP"                        
 [37] "Guide2Pharma"                "Guide2Pharma_Cellinker"     
 [39] "Guide2Pharma_LRdb"           "Guide2Pharma_talklr"        
 [41] "HPMR"                        "IntAct"                     
 [43] "ProtMapper"                  "REACH_ProtMapper"           
 [45] "Sparser_ProtMapper"          "scConnect"                  
 [47] "ACSN"                        "BioGRID"                    
 [49] "Lit-BM-17"                   "InnateDB"                   
 [51] "DOMINO"                      "CancerCellMap"              
 [53] "Reactome_LRdb"               "ELM"                        
 [55] "LMPID"                       "AlzPathway"                 
 [57] "Wojtowicz2020"               "BEL-Large-Corpus_ProtMapper"
 [59] "Macrophage"                  "I2D_CellPhoneDB"            
 [61] "InnateDB_SignaLink3"         "IMEx_CellPhoneDB"           
 [63] "InnateDB_CellPhoneDB"        "IntAct_CellPhoneDB"         
 [65] "Reactome_SignaLink3"         "InnateDB-All_CellPhoneDB"   
 [67] "Guide2Pharma_CellPhoneDB"    "HPRD_KEA"                   
 [69] "HPRD_MIMP"                   "KEA"                        
 [71] "MIMP"                        "MPPI"                       
 [73] "PhosphoNetworks"             "PhosphoPoint"               
 [75] "PhosphoSite"                 "PhosphoSite_KEA"            
 [77] "PhosphoSite_MIMP"            "PhosphoSite_ProtMapper"     
 [79] "PhosphoSite_noref"           "SIGNOR_ProtMapper"          
 [81] "iPTMnet"                     "phosphoELM"                 
 [83] "phosphoELM_KEA"              "phosphoELM_MIMP"            
 [85] "MatrixDB"                    "MINT_CellPhoneDB"           
 [87] "Adhesome"                    "DEPOD"                      
 [89] "HPRD-phos"                   "HuRI"                       
 [91] "UniProt_CellPhoneDB"         "CellChatDB-cofactors"       
 [93] "Li2012"                      "NCI-PID_ProtMapper"         
 [95] "Reactome_ProtMapper"         "RLIMS-P_ProtMapper"         
 [97] "NetworKIN_KEA"               "MatrixDB_CellPhoneDB"       
 [99] "dbPTM"                       "SPIKE_LC"                   
[101] "Kinexus_KEA"                 "Cui2007"                    
[103] "ACSN_SignaLink3"             "TCRcuration_SignaLink3"     
[105] "ARN"                         "DeathDomain"                
[107] "NRF2ome"                     "DoRothEA"                   
[109] "HTRIdb_DoRothEA"             "KEGG_DoRothEA"              
[111] "RegNetwork_DoRothEA"         "TFactS_DoRothEA"            
[113] "TRED_DoRothEA"               "TRRUST_DoRothEA"            
[115] "ORegAnno_DoRothEA"           "TRRD_DoRothEA"              
[117] "PAZAR_DoRothEA"              "ReMap_DoRothEA"             
[119] "HOCOMOCO_DoRothEA"           "JASPAR_DoRothEA"            
[121] "ARACNe-GTEx_DoRothEA"        "IntAc

In [5]:
my_source_weights_df <- 
     tibble(source = All_sources, weight = rep(1,length(All_sources)))

In [6]:
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath, 
    source_weights_df = my_source_weights_df)

In [7]:
## We read the results of the optimization 
resultsOptimization <- readRDS("NNET_Custom/Optimization_results.rds")

optimized_parameters = resultsOptimization %>% 
    process_mlrmbo_nichenet_optimization(
        source_names = my_source_weights_df$source %>% unique())

In [8]:
weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)
saveRDS(weighted_networks,"NNET_Custom/weighted_networksNonSourceWeights.rds")

In [9]:
#We now generate the matrix containing the ligand-target regulatory potential scores based on the 
#weighted integrated networks.


ligands <- as.list(unique(lr_network_Omnipath$from))

ligand_target_matrix <- construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"NNET_Custom/ligand_target_matrixNoweights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


In [11]:
## Here we also take into account the optimized source weights
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath,
    source_weights_df = optimized_parameters$source_weight_df)

weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)

ligand_target_matrix = construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"NNET_Custom/ligand_target_matrixWithweights.rds")
saveRDS(weighted_networks,"NNET_Custom/weighted_networksWithSourceWeights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


In [ ]:
lr_network_Omnipath$pair = paste(lr_network_Omnipath$from, lr_network_Omnipath$to, sep="_")